### **Quantization steps**

In [ ]:
import torch.ao.quantization.quantize_fx as quantize_fx
from tinynn.graph.quantization.quantizer import QATQuantizer

from src.models.unet_module import UNETLitModule

In [ ]:
model_ckpt = "./net/tscratch/people/plgkzaleska/ennca/depth-estimation-with-compression/logs/train/runs/2024-05-31_03-13-02/checkpoints/epoch_009.ckpt"

In [ ]:
model = UNETLitModule.load_from_checkpoint(model_ckpt)